In [82]:
import torch
import pyro
import pyro.distributions as dist
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO, JitTrace_ELBO
import pandas as pd
from torch.distributions import constraints
from pyro.infer.mcmc.api import MCMC
from pyro.infer.mcmc import NUTS

In [83]:
dict_rest = pd.read_pickle("./data/crawled_data.pickle")

In [84]:
# organize data 
ids = [d["id"] for d in dict_rest]
names = [d["name"] for d in dict_rest]
lats = [d["coordinates"]["lat"] for d in dict_rest]
lngs = [d["coordinates"]["lng"] for d in dict_rest]
ratings = [ d["rating"] for d in dict_rest]
rating_ns = [d["rating_n"] for d in dict_rest]
populartimes = [ d["populartimes"] for d in dict_rest]
poptime_sat = [ d[5]['data'] for d in populartimes]
poptime_sat_12 = [d[12] for d in poptime_sat]

In [85]:
selected_data = {'ids':ids,'names':names,'lats':lats,'lngs':lngs,\
                 'ratings':ratings,'rating_ns':rating_ns,'poptime_sat_12':poptime_sat_12}
df = pd.DataFrame(selected_data)
df_model = df[['ratings','rating_ns','poptime_sat_12']]
train_data = torch.tensor(df_model.values, dtype = torch.float)
rating, rating_ns, popularity = torch.tensor(train_data[:, 0]), torch.log(train_data[:,1]), torch.tensor(train_data[:,2])

/Users/wonjunson/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [86]:
# #Tested whether SVI runs correctly, and it did.
# rating = torch.rand(30)
# test_dist = dist.Poisson(torch.exp(rating.sum() + 1))
# popularity = torch.tensor([test_dist.sample() for i in range(len(rating))], dtype = torch.float)

In [87]:
def model(rating, rating_ns, popularity):
    beta_0 = pyro.sample("beta_0", dist.Normal(0, 1))
    beta_1 = pyro.sample("beta_1", dist.Normal(0, 1))
    beta_2 = pyro.sample("beta_2", dist.Normal(0, 1))
    lambda_ = torch.exp(beta_0 + beta_1 * rating + beta_2*rating_ns)
#     lambda_ = torch.exp(beta_0 + beta_1 * rating)
    
    with pyro.plate("data", len(popularity)):
        y = pyro.sample("obs", dist.Poisson(lambda_), obs=popularity)
    
    return y

In [77]:
def guide(rating, rating_ns, popularity):
    weights_loc = pyro.param('weights_loc', torch.randn(3))
    weights_scale = pyro.param('weights_scale', torch.ones(3), constraint=constraints.positive)        
#     weights_loc = pyro.param('weights_loc', torch.randn(2))
#     weights_scale = pyro.param('weights_scale', torch.ones(2), constraint=constraints.positive)        

    
    beta_0 = pyro.sample("beta_0", dist.Normal(weights_loc[0], weights_scale[0]))
    beta_1 = pyro.sample("beta_1", dist.Normal(weights_loc[1], weights_scale[1]))
    beta_2 = pyro.sample("beta_2", dist.Normal(weights_loc[2], weights_scale[2]))
    labmda_ = torch.exp(beta_0 + beta_1 * rating + beta_2 * rating_ns)
#     labmda_ = torch.exp(beta_0 + beta_1 * rating)



In [88]:
svi = SVI(model, 
          guide, 
          optim.Adam({"lr": .005}), 
          loss=JitTrace_ELBO(), 
          num_samples=1000)

pyro.clear_param_store()
epoch = 100000
for i in range(epoch):
    ELBO = svi.step(rating, rating_ns, popularity)
    if i % 500 == 0:
        print(ELBO)
posterior = svi.run(rating, rating_ns, popularity)


/Users/wonjunson/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:8: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  


72692.7421875
68592.6953125
27640.35546875


KeyboardInterrupt: 

In [89]:
nuts_kernel = NUTS(model)

mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200)
mcmc.run(rating, rating_ns, popularity)

hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}

warmup:   6%|▋         | 77/1200 [00:01<00:25, 44.04it/s, step size=5.92e-04, acc. prob=0.782]

KeyboardInterrupt: 

warmup:   6%|▋         | 78/1200 [00:16<00:25, 44.04it/s, step size=8.97e-04, acc. prob=0.785]